## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
  import pydrake
  import underactuated
except ImportError:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_underactuated
  setup_underactuated()

In [ ]:
import numpy as np
from IPython import get_ipython
from IPython.display import HTML

from underactuated.jupyter import setup_matplotlib_backend
plt_is_interactive = setup_matplotlib_backend()

from pydrake.all import (DiagramBuilder, PlanarSceneGraphVisualizer, SceneGraph,
                         Simulator)
from pydrake.examples.rimless_wheel import (RimlessWheel, RimlessWheelGeometry,
                                            RimlessWheelParams)

params = RimlessWheelParams()

builder = DiagramBuilder()
rimless_wheel = builder.AddSystem(RimlessWheel())
scene_graph = builder.AddSystem(SceneGraph())
RimlessWheelGeometry.AddToBuilder(
    builder, rimless_wheel.get_floating_base_state_output_port(), params,
    scene_graph)
visualizer = builder.AddSystem(
    PlanarSceneGraphVisualizer(scene_graph, xlim=[-8., 8.], ylim=[-2., 3.],
                               show=plt_is_interactive))
builder.Connect(scene_graph.get_pose_bundle_output_port(),
                visualizer.get_input_port(0))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

context = simulator.get_mutable_context()

def simulate(slope=0.08, initial_angle=0, initial_angular_velocity=5.0):
  context = diagram.CreateDefaultContext()
  diagram.GetMutableSubsystemContext(
      rimless_wheel, context).get_numeric_parameter(0).set_slope(slope)
  context.SetAccuracy(1e-4)
  context.SetContinuousState([initial_angle, initial_angular_velocity])
  simulator.reset_context(context)
  simulator.Initialize()
  
  duration = 5.0 if get_ipython() else 0.1 # sets a shorter duration during testing
  if not plt_is_interactive:
      visualizer.start_recording()
  simulator.AdvanceTo(duration)
  if not plt_is_interactive:
      ani = visualizer.get_recording_as_animation()
      HTML(ani.to_jshtml())

In [ ]:
simulate()

In [ ]:
simulate(initial_angular_velocity=5.0)

In [ ]:
simulate(initial_angular_velocity=10.0)

In [ ]:
simulate(initial_angular_velocity=0.95)

In [ ]:
simulate(initial_angular_velocity=-5.0)

In [ ]:
simulate(initial_angular_velocity=-4.8)